## RSA


In [1]:
from math import gcd
def rsa_generate_keys(p, q):
    """Generate RSA public/private key pair [1][7]"""
    n = p * q
    phi = (p-1) * (q-1) 
    # Common public exponent (65537 is standard)
    e = 65537   
    # Modular multiplicative inverse
    d = pow(e, -1, phi)  
    return (e, n), (d, n)
    
def rsa_encrypt(plaintext, public_key):
    """Encrypt using public key (e, n)"""
    e, n = public_key
    return pow(plaintext, e, n)

def rsa_decrypt(ciphertext, private_key):
    """Decrypt using private key (d, n)"""
    d, n = private_key
    return pow(ciphertext, d, n)

# Example usage
p = 61  # Prime 1
q = 53  # Prime 2
public, private = rsa_generate_keys(p, q)
message = 65  # Must be < n
cipher = rsa_encrypt(message, public)
decrypted = rsa_decrypt(cipher, private)

## Diffie Hellman

In [2]:
def diffie_hellman(prime, base, private_key):
    """Calculate shared secret [2][5]"""
    return pow(base, private_key, prime)

# Parameters (should be large in real applications)
PRIME = 23      # Shared prime
GENERATOR = 5   # Shared base

# Alice's keys
alice_private = 6
alice_public = diffie_hellman(PRIME, GENERATOR, alice_private)

# Bob's keys
bob_private = 15
bob_public = diffie_hellman(PRIME, GENERATOR, bob_private)

# Shared secret
shared_alice = diffie_hellman(PRIME, bob_public, alice_private)
shared_bob = diffie_hellman(PRIME, alice_public, bob_private)

## Caesar Cipher

In [3]:
def caesar_encrypt(text, shift):
    """Encrypt text using Caesar cipher [1][6]"""
    result = ""
    for char in text:
        if char.isupper():
            result += chr((ord(char) + shift - 65) % 26 + 65)
        elif char.islower():
            result += chr((ord(char) + shift - 97) % 26 + 97)
        else:
            result += char
    return result

def caesar_decrypt(ciphertext, shift):
    """Decrypt Caesar cipher text"""
    return caesar_encrypt(ciphertext, -shift)

# Example usage
plaintext = "Hello World"
encrypted = caesar_encrypt(plaintext, 3)  # "Khoor Zruog"
decrypted = caesar_decrypt(encrypted, 3)


## AES

In [4]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64

def pad(text):
    """PKCS#7 padding [2][7]"""
    return text + (16 - len(text) % 16) * chr(16 - len(text) % 16)

def unpad(text):
    return text[:-ord(text[-1])]

# Generate random 256-bit key and IV
key = get_random_bytes(32)
iv = get_random_bytes(16)

# Encrypt
cipher = AES.new(key, AES.MODE_CBC, iv)
plaintext = "Secret Message"
ciphertext = cipher.encrypt(pad(plaintext).encode())

# Decrypt
decrypt_cipher = AES.new(key, AES.MODE_CBC, iv)
decrypted = unpad(decrypt_cipher.decrypt(ciphertext)).decode()


ModuleNotFoundError: No module named 'Crypto'

## DES

In [ ]:
from Crypto.Cipher import DES
import hashlib

def des_encrypt(plaintext, password):
    """DES encryption with CBC mode [3][8]"""
    salt = b'\x28\xAB\xBC\xCD\xDE\xEF\x00\x33'
    key = hashlib.md5(password.encode() + salt).digest()
    iv = key[:8]
    
    cipher = DES.new(key[:8], DES.MODE_CBC, iv)
    padded = plaintext + ' ' * (8 - len(plaintext) % 8)
    return cipher.encrypt(padded.encode())

def des_decrypt(ciphertext, password):
    salt = b'\x28\xAB\xBC\xCD\xDE\xEF\x00\x33'
    key = hashlib.md5(password.encode() + salt).digest()
    iv = key[:8]
    
    cipher = DES.new(key[:8], DES.MODE_CBC, iv)
    return cipher.decrypt(ciphertext).decode().strip()


## Affine

In [ ]:
def affine_encrypt(text, a, b):
    """Encrypt using affine cipher formula: (a*x + b) mod 26 [4]"""
    return ''.join([chr(((a * (ord(c.upper()) - 65) + b) % 26) + 65) 
                   if c.isalpha() else c for c in text])

def affine_decrypt(cipher, a, b):
    """Decrypt affine cipher using modular inverse"""
    a_inv = pow(a, -1, 26)
    return ''.join([chr(((a_inv * (ord(c.upper()) - 65 - b)) % 26) + 65) 
                   if c.isalpha() else c for c in cipher])

# Example (a must be coprime with 26)
encrypted = affine_encrypt("HELLO", 3, 7)  # "HOLSM"
decrypted = affine_decrypt(encrypted, 3, 7)


## Playfair

In [ ]:
def create_playfair_square(key):
    """Generate 5x5 Playfair matrix [5]"""
    key = key.upper().replace("J", "I") + "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    matrix = []
    for char in key:
        if char not in matrix and char.isalpha():
            matrix.append(char)
    return [matrix[i:i+5] for i in range(0,25,5)]

def playfair_encrypt(plaintext, key):
    """Basic Playfair encryption structure"""
    matrix = create_playfair_square(key)
    # Full implementation requires handling letter pairs and rules
    # See reference for complete code
    return "CIPHERTEXT"

# Full implementation available at:
# https://www.nashruddin.com/playfair-cipher-in-python [5]


## Vingene

In [ ]:
def vigenere(text: str, key: str, encrypt=True):
    """Polyalphabetic cipher supporting Unicode characters"""
    result = []
    key_idx = 0
    
    for char in text:
        key_char = key[key_idx % len(key)]
        key_idx += 1
        
        if encrypt:
            cipher_char = chr((ord(char) + ord(key_char)) % 1114112)
        else:
            cipher_char = chr((ord(char) - ord(key_char)) % 1114112)
        
        result.append(cipher_char)
    
    return ''.join(result)

def vigenere_encrypt(text, key):
    return vigenere(text, key, encrypt=True)

def vigenere_decrypt(ciphertext, key):
    return vigenere(ciphertext, key, encrypt=False)

plain = "Hello World!"
key = "Leet1337:)"
cipher = vigenere_encrypt(plain, key)  # "\x94ÊÑà\xa0Sª¦¬\x95°\x86"
vigenere_decrypt(cipher, key)  # "Hello World!"


## Kasiski Attack for Vigenère

In [ ]:
def kasiski_examination(ciphertext):
    """Find repeated sequences and calculate possible key lengths"""
    seen = {}
    sequences = []
    
    for i in range(len(ciphertext)-3):
        seq = ciphertext[i:i+3]
        if seq in seen:
            sequences.append((seen[seq], i))
        else:
            seen[seq] = i
    
    # Calculate distances and find common divisors
    distances = [j - i for i, j in sequences]
    candidates = []
    for d in distances:
        for i in range(2, d+1):
            if d % i == 0 and i not in candidates:
                candidates.append(i)
    
    return sorted(candidates)

ciphertext = "your_encrypted_text_here"
key_lengths = kasiski_examination(ciphertext)
print("Likely key lengths:", key_lengths)


## Autokey Cipher

In [ ]:
ALPHA = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

def autokey_encrypt(plaintext, key):
    """Encrypt using key + plaintext as extended key"""
    key = key.upper()
    extended_key = key + plaintext
    return _autokey_cipher(plaintext, extended_key, 'encrypt')

def autokey_decrypt(ciphertext, key):
    """Decrypt using key + decrypted text"""
    key = key.upper()
    decrypted = []
    extended_key = key
    
    for i, char in enumerate(ciphertext):
        decrypted_char = _autokey_cipher(char, extended_key[i], 'decrypt')
        decrypted.append(decrypted_char)
        extended_key += decrypted_char
    
    return ''.join(decrypted)

def _autokey_cipher(text, key, mode):
    result = []
    for t, k in zip(text, key):
        if t in ALPHA:
            t_idx = ALPHA.index(t.upper())
            k_idx = ALPHA.index(k.upper())
            
            if mode == 'encrypt':
                cipher = (t_idx + k_idx) % 26
            else:
                cipher = (t_idx - k_idx) % 26
            
            result.append(ALPHA[cipher] if t.isupper() else ALPHA[cipher].lower())
        else:
            result.append(t)
    return ''.join(result)

plain = "HELLO"
key = "KEY"
cipher = autokey_encrypt(plain, key)  # "KHOQI"
autokey_decrypt(cipher, key)  # "HELLO"


## Rail Fence Cipher

In [ ]:
def rail_fence_encrypt(plaintext, rails):
    """Zigzag encryption using specified rail count"""
    rail = [['' for _ in range(len(plaintext))] for _ in range(rails)]
    dir_down = False
    row, col = 0, 0
    
    for char in plaintext:
        rail[row][col] = char
        col += 1
        
        if row == 0 or row == rails - 1:
            dir_down = not dir_down
        row += 1 if dir_down else -1
    
    return ''.join(''.join(row) for row in rail)

def rail_fence_decrypt(ciphertext, rails):
    """Reconstruct original message from ciphertext"""
    rail = [['' for _ in range(len(ciphertext))] for _ in range(rails)]
    dir_down = False
    row, col = 0, 0
    
    # Mark positions to fill
    for i in range(len(ciphertext)):
        rail[row][col] = '*'
        col += 1
        if row == 0 or row == rails - 1:
            dir_down = not dir_down
        row += 1 if dir_down else -1
    
    # Fill with ciphertext
    index = 0
    for i in range(rails):
        for j in range(len(ciphertext)):
            if rail[i][j] == '*':
                rail[i][j] = ciphertext[index]
                index += 1
    
    # Reconstruct plaintext
    plaintext = []
    row, col, dir_down = 0, 0, True
    
    for _ in range(len(ciphertext)):
        plaintext.append(rail[row][col])
        col += 1
        
        if row == 0 or row == rails - 1:
            dir_down = not dir_down
        row += 1 if dir_down else -1
    
    return ''.join(plaintext)

plain = "RAILFENCE"
cipher = rail_fence_encrypt(plain, 3)  # "RFEALECIN"
rail_fence_decrypt(cipher, 3)  # "RAILFENCE"


## Row Transposition Cipher

In [ ]:
def row_transposition_encrypt(plaintext, key):
    """Columnar transposition with specified key order"""
    key_order = {int(k): idx for idx, k in enumerate(key)}
    columns = len(key)
    
    # Split into rows
    rows = [plaintext[i*columns:(i+1)*columns] for i in range((len(plaintext)+columns-1)//columns)]
    
    # Create transposition matrix
    matrix = []
    for row in rows:
        padded = row.ljust(columns, 'X')  # Pad with 'X's
        matrix.append(padded)
    
    # Read columns in key order
    ciphertext = []
    for col in sorted(key_order.keys()):
        for row in matrix:
            if col < len(row):
                ciphertext.append(row[col])
    
    return ''.join(ciphertext)

def row_transposition_decrypt(ciphertext, key):
    """Reconstruct original message from ciphertext"""
    key_order = {int(k): idx for idx, k in enumerate(key)}
    columns = len(key)
    rows = len(ciphertext) // columns
    
    # Create matrix
    matrix = [[''] * columns for _ in range(rows)]
    idx = 0
    
    for col in sorted(key_order.keys()):
        for row in range(rows):
            if idx < len(ciphertext):
                matrix[row][col] = ciphertext[idx]
                idx += 1
    
    # Read matrix row-wise
    plaintext = []
    for row in matrix:
        plaintext.append(''.join(row).rstrip('X'))  # Remove padding
    
    return ''.join(plaintext)


plain = "HELLO WORLD"
key = "3214"
cipher = row_transposition_encrypt(plain, key)  # "HOLWELRD"
row_transposition_decrypt(cipher, key)  # "HELLOWORLD"


## Block

In [ ]:
def block_transposition_encrypt(plaintext, block_size, key):
    """Transposition within fixed-size blocks"""
    key_order = {int(k): idx for idx, k in enumerate(key)}
    num_blocks = (len(plaintext) + block_size - 1) // block_size
    
    ciphertext = []
    for block_idx in range(num_blocks):
        start = block_idx * block_size
        end = start + block_size
        block = plaintext[start:end].ljust(block_size, 'X')
        
        # Transpose block
        transposed = []
        for col in sorted(key_order.keys()):
            transposed.append(block[col])
        
        ciphertext.append(''.join(transposed))
    
    return ''.join(ciphertext)

def block_transposition_decrypt(ciphertext, block_size, key):
    """Reconstruct original message from ciphertext"""
    key_order = {int(k): idx for idx, k in enumerate(key)}
    num_blocks = len(ciphertext) // block_size
    
    plaintext = []
    for block_idx in range(num_blocks):
        start = block_idx * block_size
        end = start + block_size
        block = ciphertext[start:end]
        
        # Create matrix
        matrix = [[''] * len(key) for _ in range(block_size // len(key))]
        idx = 0
        
        for col in sorted(key_order.keys()):
            for row in range(block_size // len(key)):
                if idx < len(block):
                    matrix[row][col] = block[idx]
                    idx += 1
        
        # Read matrix row-wise
        plaintext.append(''.join(''.join(row) for row in matrix).rstrip('X'))
    
    return ''.join(plaintext)


plain = "HELLOWORLD"
block_size = 4
key = "21"
cipher = block_transposition_encrypt(plain, block_size, key)  # "HLLWOWRLD"
block_transposition_decrypt(cipher, block_size, key)  # "HELLOWORLD"


## Digital

In [ ]:
# Using cryptography library [3]
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa

private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

message = b"Important message"
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# Verification
public_key.verify(
    signature,
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)
